In [1]:
import copy
import pydotplus
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import torch
import torch.nn as nn
import torch.optim as optim

In [71]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# hyper-parameters about DNN model
input_size = 30
hidden_size = 180
num_classes = 2
# hyper-parameters about optimizer
learning_rate = 0.001
momentum = 0.9
# Hyper-parameters about training control
batch_size = 32
num_iters = 300
iters_retrain = 25
num_retrains = num_iters // iters_retrain
lambda_punish = 100 # regularization strength about DNN
epsilon_punish = 0.01 # regularization strength about surrogate model

In [62]:
class NeuralNet(nn.Module):
    
    '''Fully connected neural network with one hidden layer
    '''
    
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [63]:
class SurrogateModel(nn.Module):
    
    '''Fully connected neural network with one hidden layer
       Split the fc1 into two parts 
       because only in this way can have compute graph with DNN model weights
       so that can backpropagation to update DNN model weights and this is tree regularization
       (maybe have other ways to do this faster. Currently this is not very elegant.)
    '''
    
    def __init__(self):
        super(SurrogateModel, self).__init__()
        self.fc1_1 = nn.Linear(5400, 7500)
        self.fc1_2 = nn.Linear(360, 7500)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(7500, 1)  
    
    def forward(self, x):
        # x is the model.state_dict()
        for key, value in x.items():
            if key == 'fc1.weight':
                out1 = self.fc1_1(value.view(-1))
            elif key == 'fc2.weight':
                out2 = self.fc1_2(value.view(-1))
        out = out1 + out2
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [64]:
def get_jth_minibatach(j, batch_size, X_train, y_train):
    '''返回数据集中的第j个minibatch
       
       @param j: 第j次iters_retrain
       @param batch_size: int
       @param X_train: torch.tensor
       @param y_train: torch.tensor
    '''
    num_data = y_train.size(0)
    num_minibatches = num_data // batch_size + ((num_data % batch_size) > 0)
    j = j % num_minibatches
    start = j * batch_size
    stop = start + batch_size
    return X_train[start:stop], y_train[start:stop]

In [65]:
def get_num_weights(model):
    '''打印模型的各层weight参数个数
    '''
    for key, value in model.state_dict().items():
        if key.endswith('weight'):
            print(torch.prod(torch.tensor(value.size())))

In [66]:
def get_y_APL_train(saved_model_state_dict, X_train):
    tmp_model = NeuralNet(input_size, hidden_size, num_classes)
    tmp_model.to(device)
    y_APL_train = torch.zeros(iters_retrain)
    for i in range(len(saved_model_state_dict)):
        tmp_model.load_state_dict(saved_model_state_dict[i])
        X_train = X_train.to(device)
        outputs = tmp_model(X_train)
        _, y_pred = torch.max(outputs.data, 1)
        tree = DecisionTreeClassifier()
        X_train = X_train.to(torch.device('cpu'))
        y_pred = y_pred.to(torch.device('cpu'))
        tree.fit(X_train.numpy(), y_pred.numpy())
        decision_path_matrix = tree.decision_path(X_train.numpy())
        apl = decision_path_matrix.sum() / X_train.size(0)
        y_APL_train[i] = apl
    return y_APL_train

In [67]:
# dataset
data = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.25, random_state=2020)
X_train, X_test = torch.tensor(X_train, dtype=torch.float), torch.tensor(X_test, dtype=torch.float)
y_train, y_test = torch.tensor(y_train, dtype=torch.long), torch.tensor(y_test, dtype=torch.long)

In [74]:
# train
model = NeuralNet(input_size, hidden_size, num_classes)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
#optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# surrogate model
surrogate_model = SurrogateModel()
surrogate_model.to(device)
criterion_surrogate = nn.MSELoss()
optimizer_surrogate = optim.SGD(surrogate_model.parameters(), lr=learning_rate, momentum=momentum)
#optimizer_surrogate = optim.Adam(surrogate_model.parameters(), lr=learning_rate)
for i in range(num_retrains):
    saved_model_state_dict = [] # save the model state dict in each iters_retrain
    # train DNN model
    print('Training DNN model......')
    for j in range(iters_retrain):
        trn_x, trn_y = get_jth_minibatach(j, batch_size, X_train, y_train)
        trn_x = trn_x.to(device)
        trn_y = trn_y.to(device)
        output = model(trn_x)
        path_length = surrogate_model(model.state_dict())
        loss = criterion(output, trn_y) + lambda_punish * path_length
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        saved_model_state_dict.append(copy.deepcopy(model.state_dict()))
        if (i*iters_retrain + j + 1) % 10 == 0:
            print('DNN iters: [{0}]/[{1}] loss: {2:.2f} APL: {3:.2f}'.format((i*iters_retrain + j + 1), num_iters, 
                                                                             loss.item(), path_length.item()))
    # train Decision Tree to get {weights, APL} dataset
    y_APL_train = get_y_APL_train(saved_model_state_dict, X_train)
    print('Training surrogate model......')
    # train surrogate model
    for j in range(50):
        trn_x, trn_y = get_jth_minibatach(j, batch_size, saved_model_state_dict, y_APL_train)
        trn_y = trn_y.to(device)
        output = torch.zeros(trn_y.size(0), device=device)
        for k in range(len(trn_x)):
            output[k] = surrogate_model(trn_x[k])
        loss = criterion_surrogate(output, trn_y)
        # l2 norm
        l2_norm = 0
        for key, value in surrogate_model.state_dict().items():
            if key.endswith('weight'):
                l2_norm += value.norm()
        loss += epsilon_punish * l2_norm
        optimizer_surrogate.zero_grad()
        loss.backward()
        optimizer_surrogate.step()
        if (j+1) % 10 == 0:
            print('Surrogate iters: [{0}]/[50] loss: {1:.2f}'.format(j+1, loss.item()))

Training DNN model......
DNN iters: [10]/[300] loss: 9.58 APL: 0.09
DNN iters: [20]/[300] loss: 14.00 APL: 0.14
Training surrogate model......
Surrogate iters: [10]/[50] loss: 3.34
Surrogate iters: [20]/[50] loss: 5.81
Surrogate iters: [30]/[50] loss: 3.21
Surrogate iters: [40]/[50] loss: 3.20
Surrogate iters: [50]/[50] loss: 2.84
Training DNN model......
DNN iters: [30]/[300] loss: 693.23 APL: 6.93
DNN iters: [40]/[300] loss: 711.39 APL: 7.11
DNN iters: [50]/[300] loss: 717.72 APL: 7.17
Training surrogate model......
Surrogate iters: [10]/[50] loss: 4.05
Surrogate iters: [20]/[50] loss: 3.76
Surrogate iters: [30]/[50] loss: 3.75
Surrogate iters: [40]/[50] loss: 3.76
Surrogate iters: [50]/[50] loss: 3.75
Training DNN model......
DNN iters: [60]/[300] loss: 615.48 APL: 6.15
DNN iters: [70]/[300] loss: 616.43 APL: 6.16
Training surrogate model......
Surrogate iters: [10]/[50] loss: 2.49
Surrogate iters: [20]/[50] loss: 2.26
Surrogate iters: [30]/[50] loss: 2.19
Surrogate iters: [40]/[50]

In [75]:
# test
with torch.no_grad():
    correct = 0
    total = 0
    X_test = X_test.to(device)
    y_test = y_test.to(device)
    outputs = model(X_test)
    _, predicted = torch.max(outputs.data, 1)
    total += y_test.size(0)
    correct += (predicted == y_test).sum().item()

    print('Accuracy of the network on the Breast Cancer dataset: {0} %'.format(100 * correct / total))

Accuracy of the network on the Breast Cancer dataset: 88.1118881118881 %


In [60]:
torch.save(model.state_dict(), 'dnn_model.pth')
torch.save(surrogate_model.state_dict(), 'surrogate_model.pth')

In [21]:
# visualize
model = NeuralNet(input_size, hidden_size, num_classes)
model.to(device)
model.load_state_dict(torch.load('dnn_model.pth'))
X_train = X_train.to(device)
outputs = model(X_train)
_, y_pred = torch.max(outputs.data, 1)
tree = DecisionTreeClassifier()
X_train = X_train.to(torch.device('cpu'))
y_pred = y_pred.to(torch.device('cpu'))
tree.fit(X_train.numpy(), y_pred.numpy())
dot_data = export_graphviz(tree, out_file=None,
                           feature_names=data.feature_names,
                           class_names=data.target_names,
                           filled=True, rounded=True,
                           special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data)
graph.write_pdf('tree_visualize.pdf')

True

In [ ]:
# visualize original_tree
tree = DecisionTreeClassifier()
X_train = X_train.to(torch.device('cpu'))
y_train = y_train.to(torch.device('cpu'))
tree.fit(X_train.numpy(), y_train.numpy())
dot_data = export_graphviz(tree, out_file=None,
                           feature_names=data.feature_names,
                           class_names=data.target_names,
                           filled=True, rounded=True,
                           special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data)
graph.write_pdf('tree_visualize_unreg.pdf')